In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.metrics import classification_report

In [ ]:
xgb = pd.read_csv(r'C:\Users\loren\eclipse-workspace\BMA\resources\results_riloff\input\prediction_file\xgb_labels.csv')
new_deepmoji = pd.read_csv('C:/Users/loren/eclipse-workspace/BMA/resources/results_riloff/input/prediction_file/new_deepmoji.csv')
new_bertweet = pd.read_csv('C:/Users/loren/eclipse-workspace/BMA/resources/results_riloff/input/prediction_file/new_bertweet.csv')
transformer = pd.read_csv('C:/Users/loren/eclipse-workspace/BMA/resources/results_riloff/input/prediction_file/transformer_model_based.csv')
deepmoji = pd.read_csv('C:/Users/loren/eclipse-workspace/BMA/resources/results_riloff/input/prediction_file/deepmoji_chain.csv')
random_f = pd.read_csv('C:/Users/loren/eclipse-workspace/BMA/resources/results_riloff/input/prediction_file/rf_labels.csv')

In [ ]:
xgb = pd.read_csv(r'C:\Users\loren\eclipse-workspace\BMA\resources\results_ghosh\input\prediction\xgb_labels.csv')
new_deepmoji = pd.read_csv('C:/Users/loren/eclipse-workspace/BMA/resources/results_ghosh/input/prediction/new_deepmoji.csv')
new_bertweet = pd.read_csv('C:/Users/loren/eclipse-workspace/BMA/resources/results_ghosh/input/prediction/new_bertweet.csv')
transformer = pd.read_csv('C:/Users/loren/eclipse-workspace/BMA/resources/results_ghosh/input/prediction/transformer_model_based.csv')
deepmoji = pd.read_csv('C:/Users/loren/eclipse-workspace/BMA/resources/results_ghosh/input/prediction/deepmoji_chain.csv')
random_f = pd.read_csv(r'C:/Users/loren/eclipse-workspace/BMA/resources/results_ghosh/input/prediction/rf_labels.csv.csv')

In [ ]:
def normalize_pred(x):
    if x == '2:0':
        i = 1
    else:
        i = 0
    return i

In [ ]:
def prepare_prediction(xgb):
    xgb.predicted = xgb.predicted.apply(normalize_pred)
    xgb.distribution = xgb.distribution.str.replace(r'*', '').apply(lambda x: np.float(x))
    xgb['Unnamed: 5'] = xgb['Unnamed: 5'].str.replace(r'*', '').apply(lambda x: np.float(x))
    xgb.actual = xgb.actual.apply(normalize_pred)
    xgb_distribution = xgb[['distribution', 'Unnamed: 5']]
    return xgb, xgb_distribution

In [ ]:
xgb, xgb_distribution = prepare_prediction(xgb)
new_deemoji, new_deepmoji_distribution = prepare_prediction(new_deepmoji)
new_bertweet, new_bertweet_distribution = prepare_prediction(new_bertweet)
transformer, transformer_distribution = prepare_prediction(transformer)
deepmoji, deepmoji_distribution = prepare_prediction(deepmoji)
randomf, randomf_distribution = prepare_prediction(random_f)

## Sum probability

In [ ]:
bma_0_rf = randomf_distribution.distribution *  0.734103
bma_0_xgb = xgb_distribution.distribution *0.748976 
bma_0 = bma_0_rf + bma_0_xgb

In [ ]:
distrbution_0 = pd.concat([bma_0, new_deepmoji_distribution.distribution, new_bertweet_distribution.distribution], axis = 1)

In [ ]:
dist_0_mean = distrbution_0.sum(axis = 1)

In [ ]:
bma_1_rf = randomf_distribution['Unnamed: 5']*  0.674424
bma_1_xgb = xgb_distribution['Unnamed: 5'] *0.705880
bma_1 = bma_1_rf + bma_1_xgb 

In [ ]:
prdiction_bma = np.argmax(pd.concat([bma_0,bma_1], axis =1).values, -1)

In [ ]:
distrbution_1 = pd.concat([bma_1, new_deepmoji_distribution['Unnamed: 5'], new_bertweet_distribution['Unnamed: 5']], axis = 1) # transformer_distribution['Unnamed: 5'], deepmoji_distribution['Unnamed: 5']]

In [ ]:
dist_1_mean = distrbution_1.sum(axis = 1)

In [ ]:
predict = pd.concat([dist_0_mean, dist_1_mean], axis = 1).values

In [ ]:
print(classification_report(xgb.actual, predict.argmax(axis = -1)))

In [ ]:
print(classification_report(xgb.actual, xgb.predicted))

In [ ]:
print(classification_report(xgb.actual, new_bertweet.predicted))

In [ ]:
print(classification_report(new_deemoji.actual, new_deepmoji.predicted))

## Majority Voting

In [ ]:
clf_list = pd.concat([pd.Series(prdiction_bma, name='predicted'),new_deemoji.predicted, new_bertweet.predicted], axis=1) #,transformer.predicted, deepmoji.predicted

In [ ]:
def majority_voting(clf_list):
    prediction = []
    for i, j in clf_list.iterrows():
        voting = Counter([j[0], j[1], j[2]])
        prediction.append(voting.most_common(1)[0][0])
    prediction = np.asarray(prediction)
    
    return prediction

In [ ]:
prediction = majority_voting(clf_list)

In [ ]:
print(classification_report(xgb.actual, prediction))